In [2]:
from deap import tools, base, creator
import jax
from diversity_algorithms.controllers.fixed_structure_nn_flax import SimpleNeuralControllerFlax
import jax.numpy as jnp
from diversity_algorithms.environments.brax_env import EvaluationFunctor
from diversity_algorithms.algorithms.novelty_search import set_creator
creator.create("FitnessMax", base.Fitness, weights=(1.0,)*1)
import numpy as np
creator.create("Individual", np.ndarray, typecode="d", fitness=creator.FitnessMax)
set_creator(creator)


import numpy as np

import time
from diversity_algorithms.algorithms.utils import *
from diversity_algorithms.analysis.population_analysis import *
from diversity_algorithms.analysis.data_utils import *

from diversity_algorithms.algorithms.novelty_management import *

from diversity_algorithms.algorithms.jax_utils import *
from diversity_algorithms.environments.brax_env import create
from diversity_algorithms.environments.environments import registered_environments


from IPython.display import HTML
from brax.v1.io import html


%load_ext autoreload
%autoreload 2


# Set the parameters

In [3]:
params = {"env_name": "ant-omni",  # "ant-uni" or "ant-omni"
    	  "nb_gen": 50,
		  "pop_size": 2500,
		  "episode_length": 100,
		  "lambda_nov":500,
		  "geno_type":"realarray", 
		  "variant":"NS", 
		  "eta_m":15.0,
		  "indpb":0.1,
		  "mutpb":1,
		  "cxpb":0,
		  "min":-1,
		  "max":1,
		  "k":15,
		  "add_strategy":"novel",
		  "verbosity":"none",
		  "lambda":1
	}

# Initialise the controller and the Evalution function

In [4]:
env_params = registered_environments[params["env_name"]]
random_key = jax.random.PRNGKey(0)
controller = SimpleNeuralControllerFlax(87, 8, 2, 64)
eval_functor = EvaluationFunctor(params["env_name"], 
                                 controller=controller, 
                                 episode_length=params["episode_length"],
                                 bd_function=env_params["bd_func"],
                                 output=env_params["eval_params"]["output"])

Environment set to ant-omni


# Toolbox Initilisation

In [5]:
toolbox = base.Toolbox()
toolbox.register("population", init_pop_controller, controller=controller)
toolbox.register("mutate", mutate, eta=params["eta_m"], min_val=params["min"], max_val=params["max"], indpb=params["indpb"])
toolbox.register("mate", cxBLend, alpha=params["indpb"])
toolbox.register("map_eval", eval_functor)
v=str(params["variant"])
variant=v.replace(",","")
if (variant == "NS"): 
	toolbox.register("select", selBest, fit_attr='novelty')
elif (variant == "Fit"):
	toolbox.register("select", selBest, fit_attr='fitness')
elif (variant == "Random"):
	toolbox.register("select", random.sample)
# elif (variant == "DistExplArea"):
# 	toolbox.register("select", selBest, fit_attr='dist_to_explored_area')
else:
	print("Variant not among the authorized variants (NS, Fit, Random), assuming multi-objective variant")
	toolbox.register("select", tools.selNSGA2)

# Initialisation run

In [6]:
best_ind = []

In [7]:
lambda_ = int(params["lambda"]*params["pop_size"])
population, random_key = toolbox.population(random_key, params["pop_size"])
fit, bd, random_key = toolbox.map_eval(jnp.asarray(population), random_key)

for ind, f, b in zip(population, fit, bd):
	ind.fit = f # fit is an attribute just used to store the fitness value
	ind.parent_bd = None
	ind.bd = b
	ind.id = generate_uuid()
	ind.parent_id = None

for ind in population:
	ind.am_parent=0
 
archive=updateNovelty(population,population,None,params)
isortednov=sorted(range(len(population)), key=lambda k: population[k].novelty, reverse=True)
varian=params["variant"].replace(",","")

if ("+" in variant):
	emo=True
else:
	emo=False
 
for i,ind in enumerate(population):
	ind.rank_novelty=isortednov.index(i)
	ind.dist_to_parent=0
	if (emo): 
		if (varian == "NS+Fit"):
			ind.fitness.values = (ind.novelty, ind.fit)
		elif (varian == "NS+BDDistP"):
			ind.fitness.values = (ind.novelty, 0)
		elif (varian == "NS+Fit+BDDistP"):
			ind.fitness.values = (ind.novelty, ind.fit, 0)
		else:
			print("WARNING: unknown variant: "+variant)
			ind.fitness.values=ind.fit
	else:
		ind.fitness.values=ind.fit
	# if it is not a multi-objective experiment, the select tool from DEAP 
	# has been configured above to take the right attribute into account
	# and the fitness.values is thus ignored
 
best_ind.append(selBest(population, 1, fit_attr="fitness")[0])
gen=0	

# Learning loop

In [8]:
for gen in range(1, params["nb_gen"] + 1):
	print("Generation %d" % gen)
	t = time.time()
	offspring, random_key = varOr(random_key, population, toolbox, lambda_, params["cxpb"], params["mutpb"])
	print("varOr time: ", time.time() - t)	
	# Evaluate the individuals with an invalid fitness
	t=time.time()
	fit, bd, random_key = toolbox.map_eval(jnp.asarray(offspring), random_key)
	print("map_eval time: ", time.time() - t)

	for ind, f, b in zip(offspring, fit, bd):
		ind.fit = f
		ind.fitness.values = f
		ind.parent_bd = ind.bd
		ind.parent_id = ind.id
		ind.id = generate_uuid()
		ind.bd = b

	for ind in population:
		ind.am_parent=1

	for ind in offspring:
		ind.am_parent=0
		
	pq = population+offspring
	pop_for_novelty_estimation = pq
	t=time.time()
	archive = updateNovelty(pq, offspring, archive, params, pop_for_novelty_estimation)
	print("updateNovelty time: ", time.time() - t)
	
	# Compute the novelty rank
	nov = [ind.novelty for ind in pq]
	isortednov = np.argsort(nov)[::-1]
	rank = np.empty_like(isortednov)
	rank[isortednov] = np.arange(len(isortednov))
	
	for i,ind in enumerate(pq):
		ind.rank_novelty = rank[i]
		#print("Indiv #%d: novelty=%f rank=%d"%(i, ind.novelty, ind.rank_novelty))
		if (ind.parent_bd is None):
			ind.dist_to_parent=0
		else:
			ind.dist_to_parent=np.linalg.norm(np.asarray(ind.bd)-np.asarray(ind.parent_bd))
		if (emo):
			if (varian == "NS+Fit"):
				ind.fitness.values = (ind.novelty, ind.fit)
			elif (varian == "NS+BDDistP"):
				if (ind.parent_bd is None):
					bddistp=0
				else:
					bddistp=np.linalg.norm(np.asarray(ind.bd) - np.asarray(ind.parent_bd))
				ind.fitness.values = (ind.novelty, bddistp)
			elif (varian == "NS+Fit+BDDistP"):
				if (ind.parent_bd is None):
					bddistp=0
				else:
					bddistp=np.linalg.norm(np.asarray(ind.bd) - np.asarray(ind.parent_bd))
				ind.fitness.values = (ind.novelty, ind.fit, bddistp)
			else:
				print("WARNING: unknown variant: "+variant)
				ind.fitness.values=ind.fit
		else:
			ind.fitness.values=ind.fit
	if (verbosity(params)):
		print("Gen %d"%(gen))
	else:
		if(gen%100==0):
			print(" %d "%(gen), end='', flush=True)
		elif(gen%10==0):
			print("+", end='', flush=True)
		else:
			print(".", end='', flush=True)
	
	best_ind = pq
	# Select the next generation population
	if ("," in variant):
		population[:] = toolbox.select(offspring, params["pop_size"])		
	else:
		population[:] = toolbox.select(pq, params["pop_size"])		


Generation 1
varOr time:  1.6951560974121094
map_eval time:  0.5571110248565674
updateNovelty time:  1.6977424621582031
.Generation 2
varOr time:  0.41733407974243164
map_eval time:  0.5547072887420654
updateNovelty time:  0.6511905193328857
.Generation 3
varOr time:  0.3918731212615967
map_eval time:  0.5580000877380371
updateNovelty time:  0.6274163722991943
.Generation 4
varOr time:  0.3737757205963135
map_eval time:  0.571333646774292
updateNovelty time:  0.7798500061035156
.Generation 5
varOr time:  0.44528746604919434
map_eval time:  0.5732982158660889
updateNovelty time:  0.64176344871521
.Generation 6
varOr time:  0.41126179695129395
map_eval time:  0.5773651599884033
updateNovelty time:  0.6763021945953369
.Generation 7
varOr time:  0.3868675231933594
map_eval time:  0.5858049392700195
updateNovelty time:  0.6264829635620117
.Generation 8
varOr time:  0.4915173053741455
map_eval time:  0.6105456352233887
updateNovelty time:  0.6502435207366943
.Generation 9
varOr time:  0.3768

# Show the best individual found

In [24]:
jnp.argmax(jnp.array([ind.bd for ind in best_ind]), axis=0)

Array([2499, 2493], dtype=int32)

In [31]:
best_ind[2493].bd

Array([1.8961737, 8.148198 ], dtype=float32)

In [32]:
best = best_ind[2493]
print(best.fitness, best.bd)
best = controller.array_to_dict(best)

(14.694732666015625,) [1.8961737 8.148198 ]


In [33]:
env = create("ant-uni", episode_length=300)
state = jax.jit(env.reset)(random_key)

In [34]:
jit_step = jax.jit(env.step)
jit_inf = jax.jit(controller.predict)

In [35]:
rollout = []
while not state.done:
    rollout.append(state)
    action = jit_inf(best, state.obs)
    state = jit_step(state, action)

In [36]:
HTML(html.render(env.sys, [s.qp for s in rollout]))